<a href="https://colab.research.google.com/github/rkwi/master-thesis/blob/master/code/heat_equation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import files

import platform, sys
python_version=platform.python_version()
from distutils.version import LooseVersion, StrictVersion

if ( LooseVersion(python_version) < LooseVersion("3.0.0")):
    print("Python3 is needed!");
    print("How to fix: Runtime/Change_runtime_type/Python 3");
    sys.exit()
try:
    from dolfin import *; from mshr import *
except ImportError as e:
    !apt-get install -y -qq software-properties-common python-software-properties module-init-tools
    !add-apt-repository -y ppa:fenics-packages/fenics
    !apt-get update -qq
    !apt install -y --no-install-recommends fenics
    from dolfin import *; from mshr import *
    
import matplotlib.pyplot as plt;
from IPython.display import clear_output, display; import time; import dolfin.common.plotting as fenicsplot 
import time
import numpy as np
import pandas as pd
from fenics import *
import os, sys, shutil

dolfin_version = dolfin.__version__
print ('dolfin version:', dolfin_version)

!rm -rf * # clean up all files
# Useful commands
# Remove an empty folder      : os.rmdir("my_results")
# Remove a folder with files  : shutil.rmtree("results")
# Make a folder               : os.mkdir("my_results")
# Runtime/Change_runtime_type/Python3



In [0]:
def errornorm_L1(u_e, u):
  V = u.function_space()
  mesh = V.mesh()
  degree = V.ufl_element().degree()
  W = FunctionSpace(mesh, 'P', degree + 4)
  u_e_W = interpolate(u_e, W)
  u_W = interpolate(u, W)
  e_W = Function(W)
  e_W.vector()[:] = u_e_W.vector().get_local() - u_W.vector().get_local()
  error = abs(e_W)*dx
  return abs(assemble(error))

def errornorm_Linf(u_e, u):
  S = FunctionSpace(u.function_space().mesh(),'P',u.function_space().ufl_element().degree() + 4)
  u_e_interpolated = interpolate(u_e,S)
  u_interpolated = interpolate(u,S)
  temp = u_e_interpolated.vector() - u_interpolated.vector()
  return norm(temp,'linf')

def create_error_table(N_list, L1_error, L2_error, Linf_error):
  row = len(L1_error)
  data_array = np.zeros((row,6))
  for i in range(row):
    for j in range(6):
      if j == 0:
        data_array[i][j] = L1_error[i]
      elif j == 1 and i < row - 1:
        data_array[i][j] = np.log2(L1_error[i]/L1_error[i+1])
      elif j == 2:
        data_array[i][j] = L2_error[i]
      elif j == 3 and i < row - 1:
        data_array[i][j] = np.log2(L2_error[i]/L2_error[i+1])
      elif j == 4:
        data_array[i][j] = Linf_error[i]
      elif j == 5 and i < row - 1:
        data_array[i][j] = np.log2(Linf_error[i]/Linf_error[i+1])
  table = pd.DataFrame(data_array)
  table.set_index([[str('1/' + str(N)) for N in N_list]],inplace=True)
  table.columns = [['L1','L1_rate','L2','L2_rate','Linf','Linf_rate']]
  return table

In [0]:
N_list = [16,32,64,128]
error_L1_list = []
error_L2_list = []
error_Linf_list = []
for N in N_list:
  T = 0.5
  dt = 1.0/(2*N)
  k = Constant(dt)
  num_steps = int(T/dt)
  interpolation_degree = 3
  
  mesh = UnitSquareMesh(N, N)
  V = FunctionSpace(mesh, 'P', 1)

  def boundary(x, on_boundary):
    return on_boundary

  bc = DirichletBC(V, 0, boundary)
  
  u_e = Expression('cos(t)*sin(pi*x[0])*sin(pi*x[1])',degree=interpolation_degree, t=0)

  u0 = interpolate(u_e, V)

  u = TrialFunction(V)
  v = TestFunction(V)
  f = Expression('(-sin(t) + 2*pow(pi, 2)*cos(t))*sin(pi*x[0])*sin(pi*x[1])',degree=interpolation_degree,t=0)
  f0 = Expression('(-sin(t) + 2*pow(pi, 2)*cos(t))*sin(pi*x[0])*sin(pi*x[1])',degree=interpolation_degree,t=0)

  F = u*v*dx + 0.5*k*dot(grad(u + u0), grad(v))*dx - (u0 + 0.5*k*(f + f0))*v*dx
  a, L = lhs(F), rhs(F)

  u = Function(V)
  t = 0
  for n in range(num_steps):
    t += dt
    f.t = t
    f0.t = t - dt
    u_e.t = t

    solve(a == L, u, bc)

    u0.assign(u)
    
  error_L1_list.append(errornorm_L1(u_e,u))
  error_L2_list.append(errornorm(u_e,u,'L2'))
  error_Linf_list.append(errornorm_Linf(u_e,u))

In [0]:
u_table = create_error_table(N_list,error_L1_list,error_L2_list,error_Linf_list)

In [5]:
u_table

,L1,L1_rate,L2,L2_rate,Linf,Linf_rate
1/16,0.003978,1.992733,0.004813,1.993078,0.011062,1.991513
1/32,0.001000,1.998171,0.001209,1.998264,0.002782,1.997878
1/64,0.000250,1.999545,0.000303,1.999566,0.000696,1.999469
1/128,0.000063,0.000000,0.000076,0.000000,0.000174,0.000000


In [0]:
u_table.to_latex('u_table.txt',float_format='%.3E')